In [10]:
from selenium import webdriver
import time 
import re
import random
import requests
import logging
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import (
    NoSuchElementException, 
    ElementNotInteractableException,
    TimeoutException,
    ElementClickInterceptedException
)
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
from itertools import zip_longest


In [11]:
chrome_options = Options()
chrome_options.add_argument("--lang=en-US")
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu") 
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
service = Service()


In [12]:
def scrape(url):
    driver = webdriver.Chrome(options=chrome_options,service=service)
    driver.get(url)
    
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((
            By.XPATH, 
            '//*[contains(@data-test-attribute, "all-results-section")]'
        ))
    )
    
    try:
        driver.find_element(
            By.XPATH,
            '//button[contains(text(), "Accept")]'
        ).click()
        time.sleep(2)
    except NoSuchElementException:
        pass
    
    # Find and click "Show more" button with multiple strategies
        show_more_clicked = False
        
        # Strategy 1: Wait for element to be clickable
        try:
            show_more_button = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((
                    By.XPATH,
                    '//button//*[contains(text(), "Show more")]/ancestor::button'
                ))
            )
            show_more_button.click()
            show_more_clicked = True
            print("Show more button clicked successfully")
        except (TimeoutException, ElementNotInteractableException, ElementClickInterceptedException):
            pass
        
        # Strategy 2: Try alternative XPath
        if not show_more_clicked:
            try:
                show_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((
                        By.XPATH,
                        '//button[contains(text(), "Show more")]'
                    ))
                )
                show_more_button.click()
                show_more_clicked = True
                print("Show more button clicked with alternative XPath")
            except (TimeoutException, ElementNotInteractableException, ElementClickInterceptedException):
                pass
        
        # Strategy 3: Scroll to element and use ActionChains
        if not show_more_clicked:
            try:
                show_more_elements = driver.find_elements(
                    By.XPATH,
                    '//*[contains(text(), "Show more")]'
                )
                
                for element in show_more_elements:
                    try:
                        # Scroll to element
                        driver.execute_script("arguments[0].scrollIntoView(true);", element)
                        time.sleep(1)
                        
                        # Try clicking with ActionChains
                        actions = ActionChains(driver)
                        actions.move_to_element(element).click().perform()
                        show_more_clicked = True
                        print("Show more button clicked with ActionChains")
                        break
                    except Exception as e:
                        continue
            except Exception as e:
                print(f"ActionChains strategy failed: {e}")
        
        # Strategy 4: JavaScript click as last resort
        if not show_more_clicked:
            try:
                show_more_elements = driver.find_elements(
                    By.XPATH,
                    '//*[contains(text(), "Show more")]'
                )
                
                for element in show_more_elements:
                    try:
                        driver.execute_script("arguments[0].click();", element)
                        show_more_clicked = True
                        print("Show more button clicked with JavaScript")
                        break
                    except Exception as e:
                        continue
            except Exception as e:
                print(f"JavaScript click failed: {e}")
        
        if not show_more_clicked:
            print("Warning: Could not click 'Show more' button")
        
        # Wait for additional content to load after clicking
        time.sleep(5)
    page_source = driver.page_source
    driver.quit()
    return page_source

In [13]:
def parse(html):
    soup = BeautifulSoup(html, 'html.parser')
    listings = []
    
    for listing in soup.select('[data-test-attribute="location-results-card"]'):
        title = listing.select_one('.FGwzt')
        rating = listing.select_one('title')
        reviews = listing.select_one('[data-automation="bubbleReviewCount"]')
        href = listing.select_one('a').get('href')

        listings.append({
            'place name': title.text,
            'rating': float(rating.text.split(' ')[0]),
            'reviews': int(str(reviews.text.replace(',', '').replace('(','').replace(')','')).split(' ')[0]),
            'link': 'https://www.tripadvisor.com' + href
        })
    
    return listings


In [1]:
test=[{
    "text":"bonjour",
    "date":12,},{
    "text":"bonjour",
    "date":18
    }]

In [9]:
import pandas as pd
def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

In [ ]:
save_to_csv(test,"test.csv")

In [ ]:
# url1='https://www.tripadvisor.com/Search?q=Restaurant+in+Marrakech'
# url2='https://www.tripadvisor.com/Search?q=Tourist+Attraction+in+Marrakech'

In [ ]:
# if __name__ == '__main__':
#     html1 = scrape(url1)
#     results1 = parse(html1)
#     save_to_csv(results1, 'restaurant.csv')
#     html2=scrape(url2)
#     results2= parse(html2)
#     save_to_csv(results2, 'touristique.csv')


In [14]:
cities = ["Tetouan"]
categories = ["Hotel", "Restaurant", "Tourist Attraction"]

In [15]:
if __name__ == '__main__':
    places = []
    for city in cities:
        for cat in categories:
            cat=cat.replace(' ','+')
            query =f'{cat}+in+{city}'
            url=f'https://www.tripadvisor.com/Search?q={query}'
            html=scrape(url)
            results=parse(html)
            for i in range (len(results)):
                places.append({
                    'place name':results[i]['place name'],
                    'rating' : results[i]['rating'],
                    'reviews': results[i]['reviews'],
                    'category' : cat,
                    'city' : city,
                    'link' : results[i]['link']
                })
    save_to_csv(places ,"places_test.csv")

Show more button clicked successfully
Show more button clicked successfully
Show more button clicked successfully


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
# def scrape_reviews(url):
#      driver = webdriver.Chrome(options=chrome_options)
#      driver.set_page_load_timeout(30)
#      driver.implicitly_wait(10)
            
#      # Script anti-détection
#      driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
#      driver.get(url)
#      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#      time.sleep(2)
#      #time.sleep(random.randint(3,9))
#      WebDriverWait(driver,20).until(EC.presence_of_element_located((
#              By.XPATH, 
#              '//div[contains(@data-test-target, "review")]' 
#          ))
#      )
#      page_source = driver.page_source
#      driver.quit()
     
#      return page_source

In [ ]:
# def scrape_reviews(url):
#      driver = webdriver.Chrome(options=chrome_options)
#      driver.set_page_load_timeout(30)
#      driver.implicitly_wait(10)
#      titles=[]   
#      ratings=[] 
#      reviews_texte=[]
#      # Script anti-détection
#      driver.get(url)
#      time.sleep(2)

#      WebDriverWait(driver, 15).until(
#                 EC.element_to_be_clickable((
#                     By.XPATH,
#                     '//button//*[contains(text(), "Jump to all reviews")]/ancestor::button'
#                 ))
#             ).click()
#      reviews= WebDriverWait(driver,20).until(EC.presence_of_element_located((
#              By.XPATH, 
#              '//div[contains(@data-test-target, "review")]' 
#          ))
#      )
#      try :
#           WebDriverWait(driver,20).until(EC.presence_of_element_located((
#              By.XPATH,
#           '//*[contains(@data-test-attribute, "HR_CC_CARD")]'
#           )))
#      except Exception as e :
#           print(f"error{e}")
     
#      tilte = reviews.find_elements(By.XPATH, '//div[contains(@data-test-target, "review-title")]')
#      rating = reviews.find_elements(By.XPATH, '//div[contains(@data-test-target, "review-rating")]')
#      re_txt = reviews.find_elements(By.XPATH, '//div[contains(@data-automation,"reviewText")]')
#      for n in rating:
#           ratings.append(n.text)
#      for x in tilte :
#           titles.append(x.text)
#      for t in re_txt : 
#           reviews_texte.append(t)
#      #page_source = driver.page_source
#      driver.quit()
     
#      return reviews_texte

In [ ]:
# def scrape_reviews(url):
#     driver = webdriver.Chrome(options=chrome_options)
#     try:
#         driver.get(url)
        
#         WebDriverWait(driver, 20).until(
#             EC.element_to_be_clickable((By.XPATH, "//button[@class='rmyCe _G B- z _S c Wc wSSLS AeLHi QHaGY VauJM']"))
#         )
        

#         driver.find_element(
#                 By.XPATH,
#                 "//button[@class='rmyCe _G B- z _S c Wc wSSLS AeLHi QHaGY VauJM']"
#         ).click()
#         driver.implicitly_wait(5)
#         # scroller vers le bas
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(2)
#         page_source = driver.page_source
#         return page_source
#     finally:
#         driver.quit()

In [14]:
driver = webdriver.Chrome(options=chrome_options)
df=pd.read_csv("places.csv")
link=df.loc[0,"link"]
driver.get(f"{link}")

In [3]:
def scrape_reviews(url):   
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    results=[]
    
    try:
        driver.get(url)
        print("Page loaded, waiting for reviews...")
        
        # Try multiple selectors that TripAdvisor commonly uses
        selectors_to_try = [
            '//div[@data-test-target="HR_CC_CARD"]',
            '//div[@data-reviewid]',
            '//div[contains(@data-test-target, "review")]',
            '//section[@data-automation="reviews"]//div',
            '//*[contains(@class, "review")]'
        ]
        
        element_found = False
        for selector in selectors_to_try:
            try:
                print(f"Trying selector: {selector}")
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, selector))
                )
                print(f"Success with selector: {selector}")
                element_found = True
                break
            except TimeoutException:
                continue
        
        if not element_found:
            print("None of the selectors worked. Let's try a different approach...")
            # Wait for page to load completely
            time.sleep(5)
            
            # Check if we're on the right page
            if "tripadvisor" not in driver.current_url.lower():
                raise Exception("Not on TripAdvisor page")
            
            # Sometimes reviews load after scrolling
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)
            
            # Try waiting for any review-related element
            try:
                WebDriverWait(driver, 15).until(
                    lambda d: len(d.find_elements(By.XPATH, "//*[contains(text(), 'review') or contains(text(), 'Review')]")) > 0
                )
                print("Found review-related content")
            except TimeoutException:
                print("Could not find any review content")

        tilte = driver.find_elements(By.XPATH, '//div[contains(@data-test-target, "review-title")]')
        re_txt = driver.find_elements(By.XPATH, '//span[contains(@data-automation,"reviewText")]')
        # page_source = driver.page_source
        for x, y in zip_longest(tilte,re_txt, fillvalue=None):
            if x is not None:
                x = x.text
            else:
                x = None

            if y is not None:
                y = y.text
            else:
                y = None
            
            results.append({
                "review title": x,
                "review": y
            })
        return results
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        print(f"Current URL: {driver.current_url}")
        return None
    finally:
        driver.quit()

In [5]:
def parse_reviws(html):
    soup = BeautifulSoup(html, 'html.parser')
    reviews = []
    for review in soup.select('[data-test-target="HR_CC_CARD"]'):
        title = review.select_one('.FGwzt')
        date_text = review.select_one('.Ci')
        rating = review.select_one('title')
        review_text = review.select_one('[data-automation^="reviewText_"]')

        m = re.search(r"\b[A-Za-z]{3}\s*\d{1,2}\b", date_text.text)
        date = m.group(0) if m else None
        rating=rating.text.strip().replace(' of 5 bubbles','')
        reviews.append({
            'name': title.text,
            'rating': float(rating),
            'date' : date,
            'review text': review_text
        })
    
    return reviews

In [11]:
df_test =df[df['category']=="Hotel"]

In [12]:
index_to_drop = df_test[df_test['place name'] == "Private Chauffeur & Premium Transportation in Marrakech"].index
df_test = df_test.drop(index_to_drop)
df_test = df_test.reset_index(drop=True)

len(df_test)

29

In [4]:
url="https://www.tripadvisor.com/Hotel_Review-g293734-d1873436-Reviews-Riad_Kheirredine-Marrakech_Marrakech_Safi.html"
html=scrape_reviews(url)
# with open("page.html", "w", encoding="utf-8") as f:
#     f.write(html)
# results=parse_reviws(html)

Page loaded, waiting for reviews...
Trying selector: //div[@data-test-target="HR_CC_CARD"]
Trying selector: //div[@data-reviewid]
Trying selector: //div[contains(@data-test-target, "review")]
Success with selector: //div[contains(@data-test-target, "review")]


In [8]:
save_to_csv(html,"test.csv")

,review title,review
0,Five Star Experience!!,The Riad Kheirredine was just AMAZING!! The se...
1,The best Riad in Morocco - there aren't enough...,After spending one night in Marrakech and anot...
2,A fantastic riad in Marrakech,We're a family of four whom had a wonderful st...
3,Amazing stay at Riad Kheirredine,Its hard to describe how beautiful this place ...
4,This is the best place you will ever stay!,"Best experience of our life! I'm telling you, ..."
5,Fantasic. with a minor negative on AC.,Just like the other reviewers said. The servic...
6,A Hidden 5-Star Oasis in Marrakech,What an absolute gem of a place to stay! I was...
7,DA dream experience,We stayed in this Riad with a couple of friend...
8,That's great!,All fantastic!! You can't ask for more... from...
9,A perfect oasis in bustling Marrakesh. I would...,My wife and I just got back from a 4 night sta...


In [ ]:
url1='https://www.tripadvisor.com/Search?q=Restaurant+in+Marrakech'
html=scrape(url1)
with open("page.html", "w", encoding="utf-8") as f:
    f.write(html)


ProtocolError: ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))

In [19]:
all_data=[]
for i in range (len(df_test)):
    print(f"scraping reviwes form {df_test.loc[i,'place name']} of {df_test.loc[i,'category']} in {df_test.loc[i,'city']}")
    url=df_test.loc[i,'link']
    html=scrape_reviews(url)
    result=parse_reviws(html)
    print(f"{len(result)} of reviews scraped ")
    for j in range(len(result)):
        all_data.append({
            "place name" : df_test.loc[i,'place name'],
            "city": df_test.loc[i,'city'],
            "category" : df_test.loc[i,'category'],
            "name": result[j]['name'],
            'rating': result[j]['rating'],
            "date" : result[j]['date'],
            "review text" : result[j]['review text']
        })
    
save_to_csv(all_data ,"reviews.csv")    

scraping reviwes form Movenpick Hotel Mansour Eddahbi Marrakech of Hotel in Marrakech


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff79a4fe935+77845]
	GetHandleVerifier [0x0x7ff79a4fe990+77936]
	(No symbol) [0x0x7ff79a2b9cda]
	(No symbol) [0x0x7ff79a3106aa]
	(No symbol) [0x0x7ff79a31095c]
	(No symbol) [0x0x7ff79a363d07]
	(No symbol) [0x0x7ff79a33890f]
	(No symbol) [0x0x7ff79a360b07]
	(No symbol) [0x0x7ff79a3386a3]
	(No symbol) [0x0x7ff79a301791]
	(No symbol) [0x0x7ff79a302523]
	GetHandleVerifier [0x0x7ff79a7d684d+3059501]
	GetHandleVerifier [0x0x7ff79a7d0c0d+3035885]
	GetHandleVerifier [0x0x7ff79a7f0400+3164896]
	GetHandleVerifier [0x0x7ff79a518c3e+185118]
	GetHandleVerifier [0x0x7ff79a52054f+216111]
	GetHandleVerifier [0x0x7ff79a5072e4+113092]
	GetHandleVerifier [0x0x7ff79a507499+113529]
	GetHandleVerifier [0x0x7ff79a4ee298+10616]
	BaseThreadInitThunk [0x0x7ffd67f7e8d7+23]
	RtlUserThreadStart [0x0x7ffd6877c34c+44]


In [ ]:
test=pd.read_csv('./reviews.csv')
test[test['place name']=="Sofitel Marrakech Palais Impérial & Spa"]

In [ ]:
# soup = BeautifulSoup(html, 'html.parser')
# for listing in soup.select('[data-test-target="HR_CC_CARD"]'):
#     title = listing.select_one('.FGwzt')
#     rating = listing.select_one('title')
#     rating=rating.text.strip().replace(' of 5 bubbles','')
#     reviw_text = listing.select_one('[data-automation^="reviewText_"]')

#     print(reviw_text.text)
#     break


In [ ]:
soup = BeautifulSoup(html, 'html.parser')
for listing in soup.select('[data-test-target="HR_CC_CARD"]'):
    date_element = listing.find('div', {'class': 'tVWyV _Z o S4 H3 Ci'})
    child_divs = date_element.find_all('div')
    print(date_element)
    date = date_element.text.strip()
    print(date)